In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
from torch import nn
import torch.optim as optimzer
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader
import torchvision

CATEGORIES = ["buildings", "forest",
              "glacier", "mountain", 
              "sea", "street"]
EPOCHS = 10
TEST_SIZE = 0.30

In [63]:
def load_Intel_Image(dataDirectory : str):
    images = []
    labels = []
    for SingleCategory in CATEGORIES:
        category = os.path.join(dataDirectory, str(SingleCategory))
        
        for image in os.listdir(category):
            img_path = os.path.join(category, image)
            final_image = cv2.imread(img_path)
            
            if final_image is None:
                raise Exception("Problem with the images")
            img = cv2.resize(final_image, (64,64)) #Making the Picture 64 x 64
            images.append(img)
            labels.append(SingleCategory)
    
    return (np.array(images), np.array(labels))

            
        
            

In [64]:
images, labels = load_Intel_Image("seg_train/")

X_train, X_test, Y_train, Y_test = train_test_split(
    np.array(images), np.array(labels), test_size=TEST_SIZE, random_state=42
)

print(f"This is the shape of X_train: {X_train.shape}")
print(f"This is the shape of Y_train: {Y_train.shape}")
print(f"This is the shape of X_test: {X_test.shape}")
print(f"This is the shape of Y_Test: {Y_test.shape}")

This is the shape of X_train: (9823, 64, 64, 3)
This is the shape of Y_train: (9823,)
This is the shape of X_test: (4211, 64, 64, 3)
This is the shape of Y_Test: (4211,)


In [65]:
#1) Normalize Input Pixels
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0


#2) We need to Transpose the Matrix because TF (M, H, W, NC) -> Pytorch (M, NC, H, W)
X_train = np.transpose(X_train, (0,3,1,2))
X_test = np.transpose(X_test, (0,3,1,2))

#3) Codificate the Y to Labels Encoder
Label_Encoder = LabelEncoder()
Y_train_encoded = Label_Encoder.fit_transform(Y_train)
Y_test_encoded = Label_Encoder.fit_transform(Y_test)

#4) Convert tensor of Pytorch(The numpy array of TensorFlow)
x_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(Y_train_encoded, dtype=torch.long)
x_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(Y_test_encoded, dtype=torch.long)

In [66]:
train_Dataset = TensorDataset(x_train, y_train)
test_DataSet = TensorDataset(x_test, y_test)

train_loader = DataLoader(train_Dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_DataSet, batch_size=64, shuffle=True)


In [67]:
class CNN_Model(nn.Module):
    def __init__(self, num_classes=6):
        super(CNN_Model, self).__init__()
        
        #First Block
        self.zero_pad = nn.ZeroPad2d(3) #Padding (3,3s)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.drop1 = nn.Dropout2d(0.30)
        
        #Second Block
        self.covn2 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=6, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.drop2 = nn.Dropout2d(0.30)
        #Third Block
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=2)
        self.bn3 = nn.BatchNorm2d(32)
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=1,padding=1)
        self.drop3 = nn.Dropout2d(0.30)
        
        
        self.conv4 = nn.Conv2d(32, 16, kernel_size=2)
        self.bn4 = nn.BatchNorm2d(16)
        self.pool4 = nn.MaxPool2d(3, stride=1, padding=1)
        
        self.conv5 = nn.Conv2d(16, 8, kernel_size=1)
        self.bn5 = nn.BatchNorm2d(8)
        self.pool5 = nn.MaxPool2d(3, stride=1, padding=1)
        
        
        
        self.fc = nn.Linear(8, num_classes)
    
    
    def forward(self, x):
        #Block 1
        x = self.zero_pad(x)
        x = nn.functional.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = self.drop1(x)
        #Block 2
        x = nn.functional.relu(self.bn2(self.covn2(x)))
        x = self.pool2(x)
        x = self.drop2(x)
        #Block 3 
        x = nn.functional.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        #x = self.drop3(x)        
        
        #Block 4
        x = nn.functional.relu(self.bn4(self.conv4(x)))
        x = self.pool4(x)
        
        
        #Block 5
        x = nn.functional.relu(self.bn5(self.conv5(x)))
        x = self.pool5(x)
        
        
        #Flatten and Final Block
        x = torch.flatten(x, 1)
        x = self.fc(x)
        #Dim = 1 == np.sum() == 1

        return x
        
        
        

In [71]:
convModel = torchvision.models.resnet50(weights="IMAGENET1K_V1")
convModel.fc = nn.Linear(convModel.fc.in_features, 6)
criterion = nn.CrossEntropyLoss()
optim = optimzer.AdamW(convModel.parameters(), lr=1e-4)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/administrador/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 35.0MB/s]


In [72]:
for epoch in range(EPOCHS):
    convModel.train()
    epoch_loss = 0.0
    
    for batch_x, batch_y in train_loader:
        optim.zero_grad()
        outputs = convModel(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optim.step()
        epoch_loss += loss.item()
        
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {epoch_loss/len(train_loader):.4f}")

KeyboardInterrupt: 

In [53]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import torch


convModel.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        outputs = convModel(batch_x)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())


acc = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, average='macro')
rec = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')


print(f"✅ Validation Metrics:")
print(f"Accuracy:  {acc*100:.2f}%")
print(f"Precision: {prec*100:.2f}%")
print(f"Recall:    {rec*100:.2f}%")
print(f"F1-score:  {f1*100:.2f}%")


✅ Validation Metrics:
Accuracy:  75.49%
Precision: 75.46%
Recall:    75.48%
F1-score:  75.41%


In [54]:
def evaluate_model(model, loader, criterion, name="Validation"):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_x, batch_y in loader:
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            total += batch_y.size(0)
            correct += (preds == batch_y).sum().item()

    avg_loss = total_loss / len(loader)
    acc = correct / total
    print(f"Accuracy of the {name}: {acc:.4f}")
    print(f"Loss of the {name}: {avg_loss:.4f}")
    return acc, avg_loss

# Evaluate Model
train_acc, train_loss = evaluate_model(convModel, train_loader, criterion, "training")
test_acc, test_loss = evaluate_model(convModel, test_loader, criterion, "testing")


Accuracy of the training: 0.8818
Loss of the training: 0.3877
Accuracy of the testing: 0.7549
Loss of the testing: 0.6836
